In [2]:
import os

In [3]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        # safer access using .get() to avoid NoneType errors
        config = self.config.get("model_trainer")
        params = self.params.get("TrainingArguments")

        if config is None:
            raise ValueError("model_trainer section not found in config.yaml")
        if params is None:
            raise ValueError("TrainingArguments section not found in params.yaml")

        create_directories([config["root_dir"]])

        return ModelTrainerConfig(
            root_dir=config["root_dir"],
            data_path=config["data_path"],
            model_ckpt=config["model_ckpt"],
            num_train_epochs=params["num_train_epochs"],
            warmup_steps=params["warmup_steps"],
            per_device_train_batch_size=params["per_device_train_batch_size"],
            weight_decay=params["weight_decay"],
            logging_steps=params["logging_steps"],
            evaluation_strategy=params["evaluation_strategy"],
            eval_steps=params["eval_steps"],
            save_steps=params["save_steps"],
            gradient_accumulation_steps=params["gradient_accumulation_steps"]
        )


In [9]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk
import torch

c:\Users\aminu\miniconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        import torch
        from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
        from datasets import load_from_disk

        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load the dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Tokenize function
        def tokenize(batch):
            inputs = tokenizer(batch["dialogue"], padding="max_length", truncation=True, max_length=512)
            outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=128)
            return {
                "input_ids": inputs["input_ids"],
                "attention_mask": inputs["attention_mask"],
                "labels": outputs["input_ids"]
            }

        # Apply tokenization
        dataset_samsum_pt = dataset_samsum_pt.map(tokenize, batched=True, remove_columns=["dialogue", "summary", "id"])

        # Training arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            warmup_steps=500,
            weight_decay=0.01,
            logging_steps=10,
            save_steps=1000,
            gradient_accumulation_steps=16,
            remove_unused_columns=False,  # VERY IMPORTANT to avoid the ValueError
        )

        # Trainer
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"],
        )

        # Start training
        trainer.train()
        os.makedirs(self.config.root_dir, exist_ok=True)
        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-10-23 18:29:22,933: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-23 18:29:22,939: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-23 18:29:22,941: INFO: common: created directory at: artifacts]
[2025-10-23 18:29:22,942: INFO: common: created directory at: artifacts/model_trainer]
